In [1]:
import os
os.environ["HF_HOME"] = "/kaggle/working/hf"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

# Fix protobuf conflict first
!pip -q uninstall -y protobuf
!pip -q install -q "protobuf==4.25.3"
!python -c "import google.protobuf as pb; print('protobuf =', pb.__version__)"

# Install pinned libs (with deps)
!pip -q install -U "transformers==4.44.2" "tokenizers==0.19.1" "peft==0.11.1" \
                 "accelerate==0.33.0" "datasets==2.21.0" "sacrebleu==2.4.2"

import torch, transformers, peft, datasets
print("torch", torch.__version__)
print("transformers", transformers.__version__, "| peft", peft.__version__)
print("datasets", datasets.__version__)
print("GPU count:", torch.cuda.device_count())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
opentelemetry-proto 1.37.0 requires protobuf<7.0,>=5.0, but you have protobuf 4.25.3 which is incompatible.
a2a-sdk 0.3.10 requires protobuf>=5.29.5, but you have protobuf 4.25.3 which is incompatible.
ray 2.51.1 requires click!=8.3.0,>=7.0, but you have click 8.3.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
pydrive2 1.21.3 requires cryptography<44, but you have cryptography 46.0.3 which is incompatible.
pydrive2 1.21.3 requires pyOpenSSL<=24.2.1,>=19.1.0, but you have pyopenssl 25.3.0 which is incompatible.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have proto

In [2]:
from pathlib import Path
from datasets import Dataset, DatasetDict

PROC_DIR = Path("/kaggle/input/data-vlsp/processed")
assert PROC_DIR.exists()

TRAIN_VI, TRAIN_EN = PROC_DIR/"train.vi", PROC_DIR/"train.en"
VALID_VI, VALID_EN = PROC_DIR/"valid.vi", PROC_DIR/"valid.en"
TEST_VI,  TEST_EN  = PROC_DIR/"test.vi",  PROC_DIR/"test.en"

OUT_ROOT = Path("/kaggle/working/vlsp_vi2en_run")
OUT_ROOT.mkdir(parents=True, exist_ok=True)
DS_DIR = OUT_ROOT/"dataset_vi2en_raw"

def build_split(src_path: Path, tgt_path: Path):
    def gen():
        with src_path.open("r", encoding="utf-8") as fs, tgt_path.open("r", encoding="utf-8") as ft:
            for s, t in zip(fs, ft):
                s = s.strip()
                t = t.strip()
                if s and t:
                    yield {"src": s, "tgt": t}
    return Dataset.from_generator(gen)

raw = DatasetDict({
    "train": build_split(TRAIN_VI, TRAIN_EN),
    "valid": build_split(VALID_VI, VALID_EN),
    "test":  build_split(TEST_VI,  TEST_EN),
})
raw.save_to_disk(str(DS_DIR))
print("Saved dataset to:", DS_DIR)
print(raw)
print("Sample:", raw["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/490000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/3000 [00:00<?, ? examples/s]

Saved dataset to: /kaggle/working/vlsp_vi2en_run/dataset_vi2en_raw
DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 490000
    })
    valid: Dataset({
        features: ['src', 'tgt'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['src', 'tgt'],
        num_rows: 3000
    })
})
Sample: {'src': 'Đặc điểm của nhóm BN nghiên cứu Tổng cộng có 60 BN với 60 KHPM, bao gồm 46 nam và 14 nữ.', 'tgt': 'Characteristics of patients studied 60 patients with 60 soft-tisue defects in the weight-bearing area of the foot, including 46 male and 14 female.'}


In [3]:
from pathlib import Path

Path("/kaggle/working/train_qwen_vi2en_lora.py").write_text(r"""
import os, argparse, glob
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["TRANSFORMERS_NO_FLAX"] = "1"

import torch
from datasets import load_from_disk
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model

def make_prompt_vi2en(src_vi: str) -> str:
    return (
        "You are a professional medical translator.\n"
        "### Task: Translate Vietnamese to English (medical domain)\n"
        f"### Vietnamese: {src_vi}\n"
        "### English:"
    )

def preprocess_builder(tokenizer, max_len: int):
    eos_id = tokenizer.eos_token_id
    def _pp(ex):
        prompt = make_prompt_vi2en(ex["src"])
        completion = " " + ex["tgt"]

        prompt_ids = tokenizer(prompt, add_special_tokens=False)["input_ids"]
        comp_ids   = tokenizer(completion, add_special_tokens=False)["input_ids"]
        if eos_id is not None:
            comp_ids = comp_ids + [eos_id]

        input_ids = (prompt_ids + comp_ids)[:max_len]
        labels    = ([-100] * len(prompt_ids) + comp_ids)[:max_len]
        attn      = [1] * len(input_ids)
        return {"input_ids": input_ids, "attention_mask": attn, "labels": labels}
    return _pp

class CausalCollator:
    def __init__(self, pad_id: int):
        self.pad_id = pad_id
    def __call__(self, feats):
        max_len = max(len(f["input_ids"]) for f in feats)
        def pad(x, v): return x + [v] * (max_len - len(x))
        return {
            "input_ids": torch.tensor([pad(f["input_ids"], self.pad_id) for f in feats], dtype=torch.long),
            "attention_mask": torch.tensor([pad(f["attention_mask"], 0) for f in feats], dtype=torch.long),
            "labels": torch.tensor([pad(f["labels"], -100) for f in feats], dtype=torch.long),
        }

def last_checkpoint(output_dir: str):
    ckpts = sorted(glob.glob(os.path.join(output_dir, "checkpoint-*")), key=lambda p: int(p.split("-")[-1]))
    return ckpts[-1] if ckpts else None

def main():
    ap = argparse.ArgumentParser()
    ap.add_argument("--model_id", type=str, default="Qwen/Qwen2.5-1.5B-Instruct")
    ap.add_argument("--dataset_dir", type=str, required=True)
    ap.add_argument("--output_dir", type=str, required=True)

    ap.add_argument("--max_seq_length", type=int, default=512)
    ap.add_argument("--per_device_train_batch_size", type=int, default=1)
    ap.add_argument("--per_device_eval_batch_size", type=int, default=1)
    ap.add_argument("--gradient_accumulation_steps", type=int, default=32)

    ap.add_argument("--learning_rate", type=float, default=2e-4)
    ap.add_argument("--max_steps", type=int, default=20000)  # overnight
    ap.add_argument("--warmup_ratio", type=float, default=0.03)

    ap.add_argument("--lora_r", type=int, default=16)
    ap.add_argument("--lora_alpha", type=int, default=32)
    ap.add_argument("--lora_dropout", type=float, default=0.05)

    ap.add_argument("--eval_steps", type=int, default=2000)
    ap.add_argument("--save_steps", type=int, default=2000)
    ap.add_argument("--logging_steps", type=int, default=50)
    ap.add_argument("--seed", type=int, default=42)
    args = ap.parse_args()

    torch.manual_seed(args.seed)

    local_rank = int(os.environ.get("LOCAL_RANK", "0"))
    if torch.cuda.is_available():
        torch.cuda.set_device(local_rank)

    dsd = load_from_disk(args.dataset_dir)
    train_raw = dsd["train"]
    valid_raw = dsd["valid"]

    tok = AutoTokenizer.from_pretrained(args.model_id, use_fast=True, trust_remote_code=True)
    if tok.pad_token is None:
        tok.pad_token = tok.eos_token
    tok.padding_side = "right"

    pp = preprocess_builder(tok, args.max_seq_length)
    train_ds = train_raw.map(pp, remove_columns=train_raw.column_names, num_proc=4)
    valid_ds = valid_raw.map(pp, remove_columns=valid_raw.column_names, num_proc=1)

    model = AutoModelForCausalLM.from_pretrained(
        args.model_id,
        torch_dtype=torch.float16,
        device_map={"": local_rank} if torch.cuda.is_available() else None,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    )
    model.config.use_cache = False

    lora_cfg = LoraConfig(
        r=args.lora_r, lora_alpha=args.lora_alpha, lora_dropout=args.lora_dropout,
        bias="none", task_type="CAUSAL_LM",
        target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    )
    model = get_peft_model(model, lora_cfg)

    # IMPORTANT: keep trainable (LoRA) params in fp32 to avoid GradScaler "unscale FP16 gradients" error
    for n, p in model.named_parameters():
        if p.requires_grad:
            p.data = p.data.float()


    targs = TrainingArguments(
        output_dir=args.output_dir,
        seed=args.seed,
        fp16=True, bf16=False,

        per_device_train_batch_size=args.per_device_train_batch_size,
        per_device_eval_batch_size=args.per_device_eval_batch_size,
        gradient_accumulation_steps=args.gradient_accumulation_steps,

        learning_rate=args.learning_rate,
        warmup_ratio=args.warmup_ratio,
        max_steps=args.max_steps,

        logging_steps=args.logging_steps,
        evaluation_strategy="steps",
        eval_steps=args.eval_steps,

        save_strategy="steps",
        save_steps=args.save_steps,
        save_total_limit=3,

        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,

        report_to="none",
        ddp_find_unused_parameters=False,
        remove_unused_columns=False,
        optim="adamw_torch",

        dataloader_num_workers=4,
        dataloader_pin_memory=True,
        group_by_length=True,
    )

    trainer = Trainer(
        model=model,
        args=targs,
        train_dataset=train_ds,
        eval_dataset=valid_ds,
        data_collator=CausalCollator(tok.pad_token_id),
    )

    ckpt = last_checkpoint(args.output_dir)
    trainer.train(resume_from_checkpoint=ckpt)
    trainer.save_model(args.output_dir)
    tok.save_pretrained(args.output_dir)

if __name__ == "__main__":
    main()
""", encoding="utf-8")

print("Wrote train script.")

Wrote train script.


In [4]:
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"
TRAIN_OUT = str(OUT_ROOT/"lora_vi2en_qwen2.5_1.5b")

!TOKENIZERS_PARALLELISM=false OMP_NUM_THREADS=1 torchrun --nproc_per_node=2 /kaggle/working/train_qwen_vi2en_lora.py \
  --model_id "{MODEL_ID}" \
  --dataset_dir "{DS_DIR}" \
  --output_dir "{TRAIN_OUT}" \
  --max_seq_length 320 \
  --per_device_train_batch_size 4 \
  --per_device_eval_batch_size 4 \
  --gradient_accumulation_steps 8 \
  --learning_rate 2e-4 \
  --lora_dropout 0.05 \
  --max_steps 8000 \
  --eval_steps 800 --save_steps 800 --logging_steps 50

2025-12-15 20:28:02.728275: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-15 20:28:02.728256: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765830482.938099      95 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765830482.938082      94 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765830482.988085      94 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
E0000 00:00:1765830482.988091      95 cuda_blas.cc:1